# Computing the volume underneath a Pareto frontier
This program computes the volume under a frontier of Pareto-optimal solutions to a multi-criterion optimization model, which provides a measure of the conflict among the objectives.

In [1]:
import pandas as pd
import numpy as np
import time

### Preparation for volume computation algorithm

In [2]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_None/climateChange_EfficientSolutions_NoneOnly.csv")

In [3]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns

In [4]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [5]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [6]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0 # all objectives are min for curr example
objSenses["Northern Spotted Owl Habitat (ha)"] = 1 #except owl

In [7]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [8]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [9]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [10]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [11]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N where N is number of objectives


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Algorithm to compute volume of frontier

#### "Add-the-sides" frontier volume computation method

For a mathematical description of the below algorithm, see ./algorithmWriteUp/frontierVolumeComputation_writeup.pdf

In [12]:
# Initialize frontier volume

frontierVolume = 0

In [13]:
# Initialize sub-dimensional (N-1) volume

subDimVolume = 0

In [14]:
# Initialize the dictionary of solutions' sub-dimensional (N-1) volume contributions
# index: contribution 

subDimContributions = {}

In [15]:
# Initialize the array of boundary solutions - those not dominated in the sub-dimensional space

subDimBoundarySols = []

In [16]:
# Sort the dataframe in descending order from first column
# (always descending, since objectives normalized bad->good = 0->1)

sols = sols.sort_values(by=[objs[0]],ascending=False)

In [17]:
# Create an empty dataframe to hold the solutions whose volumes have already been accounted for,
# adding a column to hold its subdimensional volume contribution,
# and a column to indicate whether it is a boundary solution in sub-dimensional space


completedFrontierPoints = sols[sols[objs[0]]<0]
completedFrontierPoints["SubDimContribution"] = pd.Series()
completedFrontierPoints["BoundarySolution"] = pd.Series()

In [18]:
# Get list of all non-primary dimensions

dims_secondary = [col for col in sols.columns if col not in [objs[0]]]

In [19]:
def getSideVolInSubDim(dim,frontierPoint,completedFrontierPoints):
    
    sideVolInSubDim = 0
    
    otherSecondaryDims = [d for d in dims_secondary if d != dim]
    sideSols_dim = completedFrontierPoints.loc[(completedFrontierPoints["BoundarySolution"]>0) & \
                                              (completedFrontierPoints[dim] > frontierPoint[dim])]\
                                            .sort_values(by=[dim],ascending=True)
    if sideSols_dim.empty:
        return sideVolInSubDim
    
    prevDimComponent = frontierPoint[dim]
    currDimComponent = 0
    for idx,row in sideSols_dim.iterrows():
        currDimComponent = row[dim]
        dimDelta = currDimComponent - prevDimComponent
        sideVolInSubDim += dimDelta * row[otherSecondaryDims].product()
        prevDimComponent = currDimComponent
    
    return sideVolInSubDim
        

In [20]:
def getSubDimVolumeFromFrontierPoint(frontierPoint,completedFrontierPoints):
    subDimContrib = frontierPoint[dims_secondary].product()
    subDimContrib -= completedFrontierPoints["SubDimContribution"].sum()
    for dim in dims_secondary:
        subDimContrib += getSideVolInSubDim(dim,frontierPoint,completedFrontierPoints)
    
    return subDimContrib

In [21]:
def getFrontierVolume(initFrontierVolume, remainingFrontierPoints, completedFrontierPoints):
    if len(remainingFrontierPoints) == 0:
        return initFrontierVolume
    else:
        # next solution to add to frontier vol
        currSol = remainingFrontierPoints.iloc[0]
        # will always be nondominated in sub-D space
        currSol["BoundarySolution"] = True
        # change boundary status of points that the current solution dominates:
        completedFrontierPoints.loc[np.all([completedFrontierPoints[obj] < currSol[obj] for obj in dims_secondary]\
                                           ,axis=0),\
                                    "BoundarySolution"] = False
        # Get the sub-D volume for the current solution
        currSol["SubDimContribution"] = getSubDimVolumeFromFrontierPoint(currSol,completedFrontierPoints)
        # Update the frontier volume
        initFrontierVolume += currSol["SubDimContribution"] * currSol[objs[0]]
        # update the points added to the frontier
        completedFrontierPoints = completedFrontierPoints.append(currSol,ignore_index=True)
        # update the remaining points
        remainingFrontierPoints = remainingFrontierPoints.drop(currSol.name)
        # recursive call
        return getFrontierVolume(initFrontierVolume,remainingFrontierPoints,completedFrontierPoints)

In [24]:
start = time.clock()
getFrontierVolume(frontierVolume,sols,completedFrontierPoints)
end = time.clock()
print ("Elapsed time: " + (str(end-start)))

Elapsed time: 1.4155565125501397


In [85]:
newdf = sols[sols[objs[0]]>1]
newdf["subdimcontrib"] = pd.Series()
newrow = sols.iloc[22]
print(newrow.name*2+1)
newrow["subdimcontrib"] = 33
newrow["BoundarySolution"] = True
newdf = newdf.append(newrow,ignore_index=True)
#newdf = newdf.drop(0)
newdf

71


,Fire Hazard,Peak Sediment Delivery (t),Northern Spotted Owl Habitat (ha),subdimcontrib,BoundarySolution
0,0.92715,0.216589,0.954416,33.0,1.0


In [86]:
sols

,Fire Hazard,Peak Sediment Delivery (t),Northern Spotted Owl Habitat (ha)
0,1.000000,0.000000,0.453512
28,0.999815,0.016378,0.793548
39,0.998187,0.016378,0.882082
48,0.989309,0.016378,0.954416
58,0.987674,0.094724,0.736244
69,0.987528,0.061379,0.793548
78,0.986210,0.111102,0.793548
85,0.985564,0.115349,0.882082
1,0.976290,0.115349,0.954416
20,0.974407,0.148220,0.736244


In [93]:
#sols[["Fire Hazard","Peak Sediment Delivery (t)"]] < [1,1]
#pd.DataFrame([pd.Series(sols[obj] < newrow[obj]) for obj in dims_secondary])
#print(np.asarray([sols[obj] < newrow[obj] for obj in dims_secondary]))
print([sols[obj] < newrow[obj] for obj in dims_secondary])
print("-----")
print(np.all([sols[obj] < newrow[obj] for obj in dims_secondary],axis=0))

[0       True
28      True
39      True
48      True
58      True
69      True
78      True
85      True
1       True
20      True
22      True
23      True
24      True
25      True
26      True
27      True
29      True
30      True
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
40     False
41     False
42     False
43     False
       ...  
93     False
95     False
96     False
97     False
98     False
99     False
100    False
101    False
102    False
103    False
104    False
2      False
3      False
4      False
5      False
6      False
8      False
7      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
21     False
19     False
Name: Peak Sediment Delivery (t), dtype: bool, 0       True
28      True
39      True
48     False
58      True
69      True
78      True
85      True
1      False
20      True
22      True
23      True
24 